In [1]:
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
def calculate_mean_velocity(file_path):
    data = pd.read_csv(file_path, delim_whitespace=True, header=None, names=['time', 'x', 'y', 'z', 'x_smooth', 'y_smooth', 'z_smooth'])
    x_smooth = np.array(data['x_smooth'])
    y_smooth = np.array(data['y_smooth'])
    z_smooth = np.array(data['z_smooth'])
    time = np.array(data['time'])
    v_arr = np.zeros(len(data))
    t_arr = np.zeros(len(data)-1)
    for i in range(1, len(data)-1):
        t_next = time[i+1]
        x_next = x_smooth[i+1]
        y_next = y_smooth[i+1]
        z_next = z_smooth[i+1]
        t_prev = time[i-1]
        x_prev = x_smooth[i-1]
        y_prev = y_smooth[i-1]
        z_prev = z_smooth[i-1]
        r_diff = np.sqrt((x_next - x_prev)**2 + (y_next - y_prev)**2 + (z_next - z_prev)**2)
        dt = t_next - t_prev
        v_arr[i] = r_diff / dt
        t_arr[i-1] = dt
    mean_velocity = np.mean(v_arr[1:-1])  
    
    return mean_velocity , v_arr

In [4]:
def calculate_path_length(file_path):
    """
    Calculates the total disatance travelled by the bacterial track by summing the consective distances
    between points.
    Inputs - 
    file_path: take in the file to calculate this on.
    """
    # Read the data from the .txt file
    data = pd.read_csv(file_path, delim_whitespace=True, header=None,
                       names=['time', 'x', 'y', 'z', 'x_smooth', 'y_smooth', 'z_smooth'])
    
    # Extract x_smooth, y_smooth, and z_smooth for path length calculation
    x_smooth = np.array(data['x_smooth'])
    y_smooth = np.array(data['y_smooth'])
    z_smooth = np.array(data['z_smooth'])

    # Calculate the path length as the sum of the distances between consecutive points
    path_length = np.sum(np.sqrt(np.diff(x_smooth)**2 + np.diff(y_smooth)**2 + np.diff(z_smooth)**2))
    
    return path_length

FileNotFoundError: [Errno 2] No such file or directory: 'FINALDATA/WT_planktonic_final//'

In [23]:
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import random


def plot_random_tracks_3d_interactive(folder_path, num_files=5):
    # Get the list of all files in the folder
    all_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]
    # Randomly select the files
    selected_files = random.sample(all_files, num_files)
    # Initialize the figure for 3D plotting
    fig = go.Figure()

    for idx, file in enumerate(selected_files):
        file_path = os.path.join(folder_path, file)
        data = pd.read_csv(file_path, delim_whitespace=True, header=None, names=['time', 'x', 'y', 'z', 'x_smooth', 'y_smooth', 'z_smooth'])
        
        # Extract x_smooth, y_smooth, and z_smooth columns
        x_smooth = np.array(data['x_smooth'])
        y_smooth = np.array(data['y_smooth'])
        z_smooth = np.array(data['z_smooth'])
        
        # Calculate mean velocity for the current track
        mean_velocity, v_arr  = calculate_mean_velocity(file_path)
        data_length = len(v_arr)
        # Add trace for the current track
        fig.add_trace(go.Scatter3d(
            x=x_smooth,
            y=y_smooth,
            z=z_smooth,
            mode='lines+markers',
            name=f'Mean Velocity: {mean_velocity:.2f}µm/s Data length: {data_length:.2f} µm',
            line=dict(width=2),
            marker=dict(size=3)
        ))

    # Update layout
    fig.update_layout(
        width=1000, height=1000,
        scene=dict(
            xaxis_title='x (µm)', yaxis_title='y (µm)', zaxis_title='z (µm)',
            aspectmode='cube'
        ),
        title='Random Tracks from Selected Files with Mean Velocities (Interactive 3D)',
        showlegend=True
    )

    # Show the plot
    fig.show()


folder_path = 'FINALDATA/Grad_split/swimmers/WTS/' 
plot_random_tracks_3d_interactive(folder_path, num_files=20)
